In [1]:
import time
import yaml
import os
from collections import deque

from apwf.tools import fx_comp_helper, fx_daq_helper, executor_helper 
from apwf.flows import automate_helper

from funcx.sdk.client import FuncXClient
from globus_automate_client import create_flows_client


configfile = f"{os.getcwd()}/ptycho_wf_config.yml"
wf_config= yaml.safe_load(open(configfile))

apwfLogger = executor_helper.setup_logger(wf_config)

In [2]:
# Globus Online Endpoints
src_endpoint = wf_config['globus_endpoints']['aps_mona4_gcp']
dest_endpoint = wf_config['globus_endpoints']['alcf_theta']

# FuncX endpoint at ThetaGPU (ALCF) and Prisma (APS)
compute_fx_endpoint = wf_config['funcx_endpoints']['alcf_polaris_gpu']
daq_fx_endpoint = wf_config['funcx_endpoints']['aps_mona4_daq']

print(compute_fx_endpoint)

b35e121c-5ed6-4980-a32e-9aee09089c36


In [3]:
fxc = FuncXClient()

In [4]:
# Register setup functions to funcx service
daq_wf_fxid = fxc.register_function(fx_daq_helper.daq_wf_prepare)
comp_wf_fxid = fxc.register_function(fx_comp_helper.comp_wf_prepare)

## Prepare workflow environment ##
# Executor
res = executor_helper.executor_wf_prepare(wf_config['executor']['activated_iids_file_path'],
                                          wf_config['executor']['log_folder_path'],
                                          create_activated_iid_file=True, create_log_folder=True, 
                                          cleanup_activated_iid_file=False, cleanup_log_folder=False)
if res != 0: apwfLogger.error(f"Unable to setup executor folders/files: {res}")
else: apwfLogger.info("Executor folders/files are ready.")

# Data acquisition
rid = fxc.run(f"{wf_config['flow']['source']['root_folder_path']}/{wf_config['flow']['source']['input_folder_prefix']}",
              f"{wf_config['flow']['source']['root_folder_path']}/{wf_config['flow']['source']['input_position_folder_prefix']}",
              f"{wf_config['flow']['source']['root_folder_path']}/{wf_config['flow']['source']['output_folder_prefix']}",
              probe_file_path=None, create_output_folder=True, cleanup_output_folder=False,
              endpoint_id=daq_fx_endpoint, function_id=daq_wf_fxid)
res = automate_helper.fx_get_result(rid, fxc)
if res != 0: apwfLogger.error(f"Unable to setup data acquisition machine folders/files: {res}")
else: apwfLogger.info("Data acquisition machine's folders/files are ready.")

# Compute machine
rid = fxc.run(f"{wf_config['flow']['destination']['root_folder_path']}/{wf_config['flow']['destination']['input_folder_prefix']}",
              f"{wf_config['flow']['destination']['root_folder_path']}/{wf_config['flow']['destination']['output_folder_prefix']}",
              f"{wf_config['flow']['task']['executable']['params']['log_folder_path']}",
              create_input_folder=True, create_output_folder=True, create_log_folder=True, 
              cleanup_input_folder=False, cleanup_output_folder=False, cleanup_log_folder=False,
              endpoint_id=compute_fx_endpoint, function_id=comp_wf_fxid)
res = automate_helper.fx_get_result(rid, fxc)
if res != 0: apwfLogger.error(f"Unable to setup compute machine folders/files: {res}")
else: apwfLogger.info("Compute machines' folders/files are ready.")


2022-04-25 17:00:33 INFO Executor folders/files are ready.
2022-04-25 17:00:36 INFO Data acquisition machine's folders/files are ready.
2022-04-25 17:01:21 INFO Compute machines' folders/files are ready.


INFO:apwf:Compute machines' folders/files are ready.


In [5]:
# FuncX function definition for remote Polaris GPU reconstruction call
func_ptycho_uuid = fxc.register_function(fx_comp_helper.ptycho_func)
print(func_ptycho_uuid)

fc41dd8a-d056-44aa-944f-db28a2def75b


In [6]:
# FuncX function definition for remote DAQ machine directory calls
fx_func_get_file_paths_uuid = fxc.register_function(fx_daq_helper.get_file_paths)
fx_func_get_folder_paths_uuid = fxc.register_function(fx_daq_helper.get_folder_paths)
print(fx_func_get_file_paths_uuid)
print(fx_func_get_folder_paths_uuid)

6b1c87c5-4193-44f9-ad99-dc27e768a010
e125ff39-9da5-4404-9103-1d5768221f3e


In [7]:
# Globus Automate flow definition
flow_definition = automate_helper.ptycho_flow_definition

# Create a flow with the above flow definition
flows_client = create_flows_client()
flow = flows_client.deploy_flow(flow_definition, input_schema={}, title=wf_config['flow']['title'], keywords=wf_config['flow']['keywords'])
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']

In [8]:
src_config = wf_config['flow']['source']
dest_config = wf_config['flow']['destination']
task_config = wf_config['flow']['task']
exec_params_config = task_config['executable']['params']

wfc = executor_helper.EConfigs(src_config, dest_config, wf_config['flow']['only_larger_than_scanid'])

while True:
    # Get the remote folders at source input folder
    src_input_folder_paths_regex = f"{wfc.src_wf_root_path}/{wfc.src_input_folder_prefix}/*"
    rid = fxc.run(src_input_folder_paths_regex,
                  endpoint_id=daq_fx_endpoint,
                  function_id=fx_func_get_folder_paths_uuid)
    src_input_folder_paths = automate_helper.fx_get_result(rid, fxc, delay=wf_config['funcx_req_delay'])

    # Get the remote folders at source output folder
    src_output_folder_paths_regex = f"{wfc.src_wf_root_path}/{wfc.src_output_folder_prefix}/*"
    rid = fxc.run(src_output_folder_paths_regex,
                  endpoint_id=daq_fx_endpoint,
                  function_id=fx_func_get_folder_paths_uuid)
    ex_src_output_folder_paths = automate_helper.fx_get_result(rid, fxc, delay=wf_config['funcx_req_delay'])

    # Get the activated iids and find the iids that need to be processed
    activated_iids = executor_helper.read_activated_iids(wf_config['executor']['activated_iids_file_path'])
    diff_iids = executor_helper.get_unprocessed_iids(activated_iids, src_input_folder_paths, ex_src_output_folder_paths, wfc.add_larger_than)
    
    # If there is no iid/scan to be processed sleep, and then re-check 
    if(len(diff_iids)==0):
        apwfLogger.debug(f"There is no folder to be processed, "
                      f"sleeping {wf_config['data_generation_delay']} secs.")
        time.sleep(wf_config['data_generation_delay'])
        continue
    
    # There are scans to be processed, generate paths
    paths = executor_helper.WFPaths(wfc, src_input_folder_paths, diff_iids).plist
     
    # Generate flow inputs according to the paths and other configuration info.
    flow_inputs = executor_helper.WFFlowsInputs(src_endpoint, dest_endpoint, compute_fx_endpoint, 
                                            func_ptycho_uuid, task_config, exec_params_config, 
                                            wf_config['flow']['sample_name'], paths).filist

    ### Initiate/dispatch Flows ###
    q1 = deque()
    for i in range(len(flow_inputs)):
        label = str(flow_inputs[i]['iid']) + '-' + "-".join(wf_config['flow']['keywords'])
        flow_action = flows_client.run_flow(flow_id, flow_scope, flow_inputs[i], label=label)
        q1.append(flow_action)
        executor_helper.append_activated_iids([flow_inputs[i]['iid']], wf_config['executor']['activated_iids_file_path'])
        apwfLogger.info(f"{i}: Flow {flow_inputs[i]['iid']} initiated and added to q1: {flow_action['action_id']}")

Please log into Globus here:
----------------------------
https://auth.globus.org/v2/oauth2/authorize?client_id=e6c75d97-532a-4c88-b031-8584a319fa3e&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2F300f9098-a3ae-4f29-a7e9-e5bfa13b5b67%2Fflow_300f9098_a3ae_4f29_a7e9_e5bfa13b5b67_user&state=_default&response_type=code&code_challenge=4bmS2Vp--XqlL1CrKfT27-ZeEqaUobA8xY3fAZmp5Cc&code_challenge_method=S256&access_type=offline&prefill_named_grant=Globus+Automate+Command+Line+Interface+on+hostel.xray.aps.anl.gov
----------------------------

Enter the resulting Authorization Code here: 2022-04-25 17:52:51 INFO 0: Flow 1460 initiated and added to q1: b29391a9-56da-4520-aac7-d89e02277a33


INFO:apwf:0: Flow 1460 initiated and added to q1: b29391a9-56da-4520-aac7-d89e02277a33


2022-04-25 17:52:57 INFO 0: Flow 1464 initiated and added to q1: 18fdd1eb-25a1-4a3a-b16c-d2f1d0ada821


INFO:apwf:0: Flow 1464 initiated and added to q1: 18fdd1eb-25a1-4a3a-b16c-d2f1d0ada821


2022-04-25 17:52:57 INFO 1: Flow 1463 initiated and added to q1: e1eddce2-eb21-409d-8335-20f374aaa473


...

INFO:apwf:0: Flow 1597 initiated and added to q1: 262f2389-b365-434b-afc1-3f6b0b55739b


2022-04-28 08:10:36 INFO 0: Flow 1598 initiated and added to q1: c20b558b-0b7e-48c7-a39f-d60be28641d6


INFO:apwf:0: Flow 1598 initiated and added to q1: c20b558b-0b7e-48c7-a39f-d60be28641d6


2022-04-28 08:10:46 INFO 0: Flow 1599 initiated and added to q1: 4ad80581-7fb6-47d3-98a3-af8942725543


INFO:apwf:0: Flow 1599 initiated and added to q1: 4ad80581-7fb6-47d3-98a3-af8942725543


In [ ]:
# Cancel all flows
for flow_action in q1:
    flows_client.flow_action_cancel(flow_id, flow_scope, flow_action['action_id'])